This started as a tangent on which **Powershell** concept is the most like [`Extension methods` in `c#`](https://learn.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/extension-methods)

The main requirements are

- It must add methods to existing types, *without* creating a new derived type
- And must not modify the original type either.
- Finally calling methods should act like they were instance methods on the extended type

`Update-TypeData` changes the original type, we want to modify the type for one instance. [Add-Member](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.utility/add-member?view=powershell-7) will do that.

What does this show? 
 
- `$folder`'s definition has a script method (ie: instance method) named `Crumbs`
- It acts like an instance method, and 
- The existing type of `[IO.DirectoryInfo]` was not changed (that was the goal)

In [52]:
gi . | % FullName | Join-String -op 'base path: ' -os "`n"

$Folder = Get-Item . 
  $Folder | Add-Member -MemberType ScriptMethod -Name 'Crumbs' -value { 
   param([int]$Limit)

   $crumbs = $this.FullName -split '\\'
   $select = if( $Limit ) { $crumbs | select -last $Limit } else { $Crumbsbr } 
   $prefix = if( $select.count -lt $crumbs.count ) { "$($this.PSDrive)://…/" } else {''}
   $select | Join-String -Separator '/' -op $prefix
}

# Html Style Newline `</br>`. this works, but the notebook is truncating extra lines
function Br { param( [int]$Count = 1) '' * $count }

$folder.FullName
br

$folder.Crumbs(2)
br

$folder.Crumbs(3)
br

$folder.Crumbs(5)

base path: C:\Users\cppmo_000\SkyDrive\Documents\2022\My_Github\notebooks\Pwsh\Types

C:\Users\cppmo_000\SkyDrive\Documents\2022\My_Github\notebooks\Pwsh\Types

C://…/Pwsh/Types

C://…/notebooks/Pwsh/Types

C://…/2022/My_Github/notebooks/Pwsh/Types


In [50]:
$AnotherFolder = gi .
$AnotherFolder.Crumbs()

InvalidOperation: 
Line |
   2 |  $AnotherFolder.Crumbs()
     |  ~~~~~~~~~~~~~~~~~~~~~~~
     | Method invocation failed because [System.IO.DirectoryInfo] does not contain a method named 'Crumbs'.


| Input | Value | 
| - | - | 
| $file.FullName | C:\Users\nin\Documents\2022\My_Github\notebooks\Pwsh\Types |
| $file.Crumbs() | C:/Users/nin/Documents/2022/My_Github/notebooks/Pwsh/Types |
| $file.Crumbs(2) |  C://…/Pwsh/Types | 
| $file.Crumbs(3) |  C://…/notebooks/Pwsh/Types | 
| $file.Crumbs(5) |  C://…/2022/My_Github/notebooks/Pwsh/Types | 
```

